In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data_8_day()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_bnd3,wp_bnd7,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,0.032962,0.136763,0.396594,0.353988,0.641934,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,0.032200,0.131800,0.407211,0.376832,0.654545,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,0.031425,0.128250,0.416158,0.394189,0.664269,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,0.030650,0.124700,0.425106,0.411547,0.673993,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,0.029875,0.121150,0.434053,0.428904,0.683716,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1030 entries, 0 to 1029
Data columns (total 28 columns):
PET             1030 non-null float64
VPD             1030 non-null float64
air_temp        1030 non-null float64
doy             1030 non-null int64
precip          1030 non-null float64
soil_temp       1030 non-null float64
sw_in           1030 non-null float64
wind_speed      1030 non-null float64
year            1030 non-null int64
wp_RNET         1030 non-null float64
wp_ch4_gf       1030 non-null float64
wp_co2_gf       1030 non-null float64
wp_er           1030 non-null float64
wp_gpp          1030 non-null float64
wp_h            1030 non-null float64
wp_le           1030 non-null float64
wp_bnd1         1030 non-null float64
wp_bnd2         1030 non-null float64
wp_bnd3         1030 non-null float64
wp_bnd7         1030 non-null float64
wp_evi          1030 non-null float64
wp_lswi2        1030 non-null float64
wp_ndvi         1030 non-null float64
wp_LST.day      1030 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_RNET"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1030, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.970661658993
CV Scores:  0.918085068953, 0.979543719853, 0.984492728162, 0.979950871214, 0.975472971384, 0.980005824747, 0.970603110849, 0.984156360045, 0.975672417445, 0.958633517276
OOB score: 0.971188445494
Feature Importances:
('sw_in', 0.32591332808692253)
('PET', 0.20887063224947544)
('wp_LST.day', 0.16873005504783717)
('wp_LST.night', 0.11891603594962881)
('air_temp', 0.050109700098541078)
('VPD', 0.048121874074752664)
('wind_speed', 0.021714158039798988)
('wp_ndvi', 0.018388054764583935)
('wp_evi', 0.015668371320290888)
('wp_lswi2', 0.011369650105997524)
('LW_IN.wp', 0.0070953641837920925)
('precip', 0.0051027760783787122)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.971190272561
CV Scores:  0.915173771372, 0.978186601411, 0.988149022609, 0.985597276267, 0.979812262802, 0.976557749267, 0.966489433057, 0.984206419612, 0.977799405151, 0.959930784064
Feature Importances:
('sw_in', 0.19399226423521437)
('wp_LST.day', 0.10925732284218222)
('VPD', 0.10621399440090931)
('LW_IN.wp', 0.097585177300486659)
('wp_LST.night', 0.08970883198846924)
('air_temp', 0.071663201160500276)
('wp_evi', 0.071034003231201473)
('PET', 0.067126777362266382)
('wp_lswi2', 0.066333724261312219)
('wp_ndvi', 0.052271180418044294)
('wind_speed', 0.048882392152982179)
('precip', 0.025931130646431298)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.964270484672
CV Scores:  0.884899094662, 0.983580470366, 0.983348447349, 0.983342204966, 0.980807626076, 0.969869065003, 0.955740686296, 0.970119844264, 0.970947843421, 0.960049564315


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 174.80829
Step #501, epoch #50, avg. loss: 3.83718
Step #1001, epoch #100, avg. loss: 1.74007
Step #1501, epoch #150, avg. loss: 1.46212
Step #2001, epoch #200, avg. loss: 1.25572
Step #2501, epoch #250, avg. loss: 1.04713
Step #3001, epoch #300, avg. loss: 0.90134
Step #3501, epoch #350, avg. loss: 0.69172
Step #4001, epoch #400, avg. loss: 0.59503
Step #4501, epoch #450, avg. loss: 0.47905
Step #1, avg. loss: 173.74677
Step #501, epoch #50, avg. loss: 3.73998
Step #1001, epoch #100, avg. loss: 1.64200
Step #1501, epoch #150, avg. loss: 1.32266
Step #2001, epoch #200, avg. loss: 1.04820
Step #2501, epoch #250, avg. loss: 0.83199
Step #3001, epoch #300, avg. loss: 0.66329
Step #3501, epoch #350, avg. loss: 0.43777
Step #4001, epoch #400, avg. loss: 0.36483
Step #4501, epoch #450, avg. loss: 0.25787
Step #1, avg. loss: 182.04988
Step #501, epoch #50, avg. loss: 3.85968
Step #1001, epoch #100, avg. loss: 1.70690
Step #1501, e

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x10c670488>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)